In [5]:
#from my_snn.rsnn import RSNN, RSNN_2l, RSNN_2l_ALIF
from my_snn.rsnn_delays import RSNN_d_d, RSNN_d_i, RSNN_d_all
from my_snn.tonic_dataloader import DatasetLoader
from my_snn.utils import train, training_plots, ModelLoader
import torch
device = "cuda:0"

import matplotlib.pyplot as  plt

time_window = 250
batch_size = 256
dataset = 'shd'

DL = DatasetLoader(dataset=dataset, caching='disk', num_workers=0, batch_size=batch_size, time_window=time_window)
test_loader, train_loader = DL.get_dataloaders()

ckpt_dir = 'barridos_shd'

data = test_loader, train_loader

dropout = 0.2

import numpy as np
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader, dropout=dropout)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

In [6]:
model_48_10_name = 'shd250_48x2_d_150_10_fs_8779296875'
model_64_10_name = 'shd250_64x2_d_150_10_fs_89013671875'
model_128_10_name = 'shd250_128x2_d_150_10_fs_89697265625'

In [9]:
for name in [model_48_10_name, model_64_10_name]:
    for k in [1, 5, 10]:
        snn_pooled = ModelLoader(name, 'shd_soa_baselines', batch_size, 'cuda:0')
        snn_pooled.pool_delays('ho', k=k)
        snn_pooled.test(test_loader, dropout = dropout)
        snn_pooled.train_loss = []
        snn_pooled.test_loss = []
        snn_pooled.acc = []
        snn_pooled.modelname = f'{name}_pooled_{k}'
        train(snn_pooled, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
        torch.cuda.empty_cache()     

delays: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 2.9737496376037598
Avg spk_count per neuron for all 250 timesteps 13.888940811157227
Test Accuracy of the model on the test samples: 24.170
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=48, bias=False)
  (f1_f2): Linear(in_features=720, out_features=48, bias=False)
  (f2_o): Linear(in_features=720, out_features=20, bias=False)
)
Total params: 82560
Total mult-adds (M): 20.64
training shd250_48x2_d_150_10_fs_8779296875_pooled_1 for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 24.42811
Step [20/31], Loss: 22.70258
Step [30/31], Loss: 22.05023
Time elasped: 57.04357051849365
Epoch [2/50]
Step [10/31], Loss: 20.86560
Step [20/31], Loss: 20.91264
Step [30/31], Loss: 20.34876
Time elasped: 56.84964084625244
Epoch [3/50]
Step [10/31], Loss: 19.52209
Step [20/31], Loss: 19.43220
Step [30/31], Loss: